In [1]:
# import os
import sys

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")
import numpy as np
import datetime

from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup
from tudatpy.kernel.numerical_simulation.estimation_setup import observation

# from astropy.time import Time
from astropy.coordinates import EarthLocation
from astropy import units as u

# from pac.data import BatchMPC
from tudatpy.data.mpc import BatchMPC
# import tudatpy.data as data

import matplotlib.pyplot as plt



In [2]:
# SPICE KERNELS
spice.load_standard_kernels()
spice.load_kernel(r"codes_300ast_20100725.bsp")
spice.load_kernel(r"codes_300ast_20100725.tf")

In [3]:
codes = [1]

batch = BatchMPC()
batch.get_observations(codes)
batch.filter(
    epoch_start=datetime.datetime(2016, 1, 1),
    # epoch_start=708405948+10000,
    epoch_end=datetime.datetime(2023, 7, 1),
    observatories_exclude=["C51", "C59", "C57"],
)

batch.summary()

print(batch.observatories_table(only_in_batch=True, only_space_telescopes=True, include_positions=False))


   Batch Summary:
1. Batch includes 1 minor planets:
   ['1']
2. Batch includes 206 observations, including 0 observations from space telescopes
3. The observations range from 2016-05-15 03:56:28.608013 to 2023-06-14 13:23:15.993581
   In seconds TDB since J2000: 516556656.7932397 to 740021065.178199
   In Julian Days: 2457523.66422 to 2460110.057824
4. The batch contains observations from 30 observatories, including 0 space telescopes

Empty DataFrame
Columns: [Code, Name, count]
Index: []


In [4]:
bodies_to_propagate = batch.MPC_objects
central_bodies = ["SSB" for _ in batch.MPC_objects]
bodies_to_create = [
    "Sun",
    "Mercury",
    "Venus",
    "Earth",
    "Moon",
    "Mars",
    "Jupiter",
    "Saturn",
    "Uranus",
    "Neptune",
]

accelerations = {
    "Sun": [
        propagation_setup.acceleration.point_mass_gravity(),
        propagation_setup.acceleration.relativistic_correction(True),
    ],
    "Mercury": [propagation_setup.acceleration.point_mass_gravity()],
    "Venus": [propagation_setup.acceleration.point_mass_gravity()],
    "Earth": [propagation_setup.acceleration.point_mass_gravity()],
    "Moon": [propagation_setup.acceleration.point_mass_gravity()],
    "Mars": [propagation_setup.acceleration.point_mass_gravity()],
    "Jupiter": [propagation_setup.acceleration.point_mass_gravity()],
    "Saturn": [propagation_setup.acceleration.point_mass_gravity()],
    "Uranus": [propagation_setup.acceleration.point_mass_gravity()],
    "Neptune": [propagation_setup.acceleration.point_mass_gravity()],
}



In [5]:
global_frame_origin = "SSB"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation
)

# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [6]:
observation_collection, links = batch.to_tudat(bodies=bodies)
observation_settings_list = list()
for link in list(links.values()):
    observation_settings_list.append(observation.angular_position(link))

In [7]:
end = batch.epoch_end
start = batch.epoch_start

In [8]:
acceleration_settings = {}
for body in batch.MPC_objects:
    acceleration_settings[str(body)] = accelerations

acceleration_models = propagation_setup.create_acceleration_models(
    bodies, acceleration_settings, bodies_to_propagate, central_bodies
)

In [9]:
initial_states = spice.get_body_cartesian_state_at_epoch("Ceres", "SSB", "J2000", "NONE", end)

# # Add random initial states
# initial_states[0:3] += + np.random.rand(3)*1000e3
# initial_states[3:] += + np.random.rand(3)*10

In [10]:
termination_condition = propagation_setup.propagator.time_termination(start)


dt = -60000
# Create numerical integrator settings
integrator_settings = propagation_setup.integrator.runge_kutta_variable_step_size(
    end, dt, propagation_setup.integrator.rkf_78, dt, dt, 1.0, 1.0
)

# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies=central_bodies,
    acceleration_models=acceleration_models,
    bodies_to_integrate=bodies_to_propagate,
    initial_states=initial_states,
    initial_time=end,
    integrator_settings=integrator_settings,
    termination_settings=termination_condition,
)

In [11]:
# Setup parameters settings to propagate the state transition matrix
parameter_settings = estimation_setup.parameter.initial_states(
    propagator_settings, bodies
)

# Create the parameters that will be estimated
parameters_to_estimate = estimation_setup.create_parameter_set(
    parameter_settings, bodies, propagator_settings
)

print(central_bodies)
print(bodies_to_propagate)

['SSB']
['1']


In [12]:

estimator = numerical_simulation.Estimator(
    bodies=bodies,
    estimated_parameters=parameters_to_estimate,
    observation_settings=observation_settings_list,
    propagator_settings=propagator_settings,
    integrate_on_creation=True, 
)


pod_input = estimation.EstimationInput(
    observations_and_times=observation_collection,
    convergence_checker=estimation.estimation_convergence_checker(
        maximum_iterations=20,
    )
)

In [13]:
pod_input.define_estimation_settings(reintegrate_variational_equations=True)


In [14]:
pod_output = estimator.perform_estimation(pod_input)
results = parameters_to_estimate.parameter_vector
results = pod_output.parameter_history[:, -1]

result_ceres = results[0:6]



Calculating residuals and partials 412
Parameter update   -230572    -228903    37632.6 -0.0035282 0.00509285  0.0100076
Current residual: 4.87632e-06
Calculating residuals and partials 412
Parameter update    0.250093     -1.01803     0.166031   1.9354e-09   3.3845e-08 -5.99652e-08
Current residual: 3.7795e-06
Calculating residuals and partials 412
Parameter update  -0.0101704   0.00364215   0.00358241 -5.28576e-10  8.61629e-11  1.69324e-10
Current residual: 3.7795e-06
Calculating residuals and partials 412
Parameter update 0.000608549  -0.00190114  -0.00100505  9.50127e-11  1.80298e-11 -2.77337e-11
Current residual: 3.7795e-06
Calculating residuals and partials 412
Parameter update  0.00147489   -0.0038384  -0.00162737  8.92955e-11   1.7105e-11 -1.27761e-11
Current residual: 3.7795e-06
Calculating residuals and partials 412
Parameter update -0.00282866    0.0023803   0.00160794 -1.83668e-10  8.74678e-12  5.55285e-11
Current residual: 3.7795e-06
Calculating residuals and partials 412


In [18]:
print(f"Ceres radial error: {np.sqrt(np.square((np.array(result_ceres) - initial_states)[0:3]).sum())/1000} km")

print(observation_collection.observation_vector_size)
print(len(observation_collection.concatenated_observations))
print(len(observation_collection.concatenated_times))

Ceres radial error: 327.0726319863498 km
412
412
412
